In [1]:
import os
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.rcParams['figure.dpi'] = 150
from nflows import transforms, distributions, flows
import torch.optim as optim
from nflows.utils import torchutils
from data import get_dataloader
import h5py
torch.set_default_dtype(torch.float64)

particle = 'piplus'

preproc = 'log10'


In [2]:
# folder containing the GEANT data used to train/validate Caloflow
trg_data_folder = '/home/yp325/regression_project/regression_with_CF/hdf5-helper'

discrete_folder = '/home/yp325/regression_project/data_discrete_100k'

In [3]:
device = torch.device('cuda:0')

In [4]:
# utilities, some are no longer used with log10 preproc
ALPHA = 1e-6
def logit(x):
    """ returns logit of input """
    return torch.log(x / (1.0 - x))

def logit_trafo(x):
    """ implements logit trafo of MAF paper https://arxiv.org/pdf/1705.07057.pdf """
    local_x = ALPHA + (1. - 2.*ALPHA) * x
    return logit(local_x)

def inverse_logit(x, clamp_low=0., clamp_high=1.):
    """ inverts logit_trafo(), clips result if needed """
    return ((torch.sigmoid(x) - ALPHA) / (1. - 2.*ALPHA)).clamp_(clamp_low, clamp_high)

def trafo_to_unit_space(energy_array):
    """ transforms energy array to be in [0, 1] """
    num_dim = len(energy_array[0])-2
    ret = [(torch.sum(energy_array[:, :-1], dim=1)/energy_array[:, -1]).unsqueeze(1)]
    for n in range(num_dim):
        ret.append((energy_array[:, n]/energy_array[:, n:-1].sum(dim=1)).unsqueeze(1))
    return torch.cat(ret, 1).clamp_(0., 1.)

def trafo_to_energy_space(unit_array, etot_array):
    """ transforms unit array to be back in energy space """
    assert len(unit_array) == len(etot_array)
    num_dim = len(unit_array[0])
    unit_array = torch.cat((unit_array, torch.ones(size=(len(unit_array), 1)).to(unit_array.device)), 1)
    ret = [torch.zeros_like(etot_array)]
    ehat_array = unit_array[:, 0] * etot_array
    for n in range(num_dim):
        ret.append(unit_array[:, n+1]*(ehat_array-torch.cat(ret).view(
            n+1, -1).transpose(0, 1).sum(dim=1)))
    ret.append(etot_array)
    return torch.cat(ret).view(num_dim+2, -1)[1:].transpose(0, 1)

class RandomPermutationLayer(transforms.Permutation):
    """ Permutes elements with random, but fixed permutation. Keeps pixel inside layer. """
    def __init__(self, features, dim=1):
        """ features: list of dimensionalities to be permuted"""
        assert isinstance(features, list), ("Input must be a list of integers!")
        permutations = []
        for index, features_entry in enumerate(features):
            current_perm = np.random.permutation(features_entry)
            if index == 0:
                permutations.extend(current_perm)
            else:
                permutations.extend(current_perm + np.cumsum(features)[index-1])
        super().__init__(torch.tensor(permutations), dim)

class InversionLayer(transforms.Permutation):
    """ Inverts the order of the elements in each layer.  Keeps pixel inside layer. """
    def __init__(self, features, dim=1):
        """ features: list of dimensionalities to be inverted"""
        assert isinstance(features, list), ("Input must be a list of integers!")
        permutations = []
        for index, features_entry in enumerate(features):
            current_perm = np.arange(features_entry)[::-1]
            if index == 0:
                permutations.extend(current_perm)
            else:
                permutations.extend(current_perm + np.cumsum(features)[index-1])
        super().__init__(torch.tensor(permutations), dim)

def add_noise(input_tensor,noiseamount=1e-8):
    noise = np.random.rand(*input_tensor.shape)*noiseamount
    return input_tensor+noise

def add_fake_noise(input_tensor,noiseamount=1e-8):
    noise = noiseamount/2
    return input_tensor+noise

# Define and load layer energy model

In [5]:
class RegressionLayerNet(nn.Module):
    def __init__(self):
        super(RegressionLayerNet, self).__init__()
        self.fc1 = nn.Linear(6, 256)  
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 1)  

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

## Train layer energy model 

In [6]:
kwargs = {'num_workers': 2, 'pin_memory': True} if device.type is 'cuda' else {}
batch_size = 200
num_epochs = 200

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_475586/3665156468.py:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  kwargs = {'num_workers': 2, 'pin_memory': True} if device.type is 'cuda' else {}


In [7]:
train_dataloader, val_dataloader = get_dataloader('piplus',
                                                           trg_data_folder,
                                                           full=False,
                                                           apply_logit=False,
                                                           device=device,
                                                           batch_size=batch_size,
                                                           with_noise=False,
                                                           normed=False,
                                                           normed_layer=False) #change dataset as appropriate

In [8]:
model_dir = "MSE_uniform_models_large/" #change as appropriate

In [11]:
for instance in range(10):
    # Create an instance of the regression model
    model = RegressionLayerNet().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    lr_schedule = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                        milestones=[15, 50, 100, 150],
                                                        gamma=0.5,
                                                        verbose=True)
    best_model_path = model_dir+'best_model_uniform_'+str(instance)+'.pth' #change as appropriate
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        for i, batch in enumerate(train_dataloader):
            model.train()
            energy = batch['energy']
            E0 = batch['layer_0_E']
            E1 = batch['layer_1_E']
            E2 = batch['layer_2_E']
            E3 = batch['layer_3_E']
            E4 = batch['layer_4_E']
            E5 = batch['layer_5_E']

            y = torch.log10(energy*10.).to(device)
            x = torch.cat((E0.unsqueeze(1),
                        E1.unsqueeze(1),
                        E2.unsqueeze(1),
                        E3.unsqueeze(1),
                        E4.unsqueeze(1),
                        E5.unsqueeze(1)), 1).to(device)

            outputs = model(x)
            loss = criterion(outputs, y)

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        with torch.no_grad():
            model.eval()
            val_losses = []
            for i, batch in enumerate(val_dataloader):
                energy = batch['energy']
                E0 = batch['layer_0_E']
                E1 = batch['layer_1_E']
                E2 = batch['layer_2_E']
                E3 = batch['layer_3_E']
                E4 = batch['layer_4_E']
                E5 = batch['layer_5_E']

                y_val = torch.log10(energy*10.).to(device)
                x_val = torch.cat((E0.unsqueeze(1),
                            E1.unsqueeze(1),
                            E2.unsqueeze(1),
                            E3.unsqueeze(1),
                            E4.unsqueeze(1),
                            E5.unsqueeze(1)), 1).to(device)

                val_outputs = model(x_val)
                val_loss = criterion(val_outputs, y_val)
                val_losses.append(val_loss.item())
            overall_val_loss = np.array(val_losses)

            val_loss_mean = overall_val_loss.mean(0)
            val_loss_std = np.sqrt(overall_val_loss.var(0))

        lr_schedule.step()
        output = 'Evaluate (epoch {}) -- '.format(epoch+1) +\
                    'MSE loss = {:.6f} +/- {:.6f}'
        if val_loss_mean < best_val_loss:
            best_val_loss = val_loss_mean
            # Save the best model checkpoint
            torch.save(model.state_dict(), best_model_path)
        print(output.format(val_loss_mean, val_loss_std))



Adjusting learning rate of group 0 to 1.0000e-04.
Adjusting learning rate of group 0 to 1.0000e-04.
Evaluate (epoch 1) -- MSE loss = 0.014668 +/- 0.004336
Adjusting learning rate of group 0 to 1.0000e-04.
Evaluate (epoch 2) -- MSE loss = 0.006073 +/- 0.001972
Adjusting learning rate of group 0 to 1.0000e-04.
Evaluate (epoch 3) -- MSE loss = 0.004477 +/- 0.001555
Adjusting learning rate of group 0 to 1.0000e-04.
Evaluate (epoch 4) -- MSE loss = 0.004046 +/- 0.001421
Adjusting learning rate of group 0 to 1.0000e-04.
Evaluate (epoch 5) -- MSE loss = 0.003758 +/- 0.001297
Adjusting learning rate of group 0 to 1.0000e-04.
Evaluate (epoch 6) -- MSE loss = 0.003920 +/- 0.001212
Adjusting learning rate of group 0 to 1.0000e-04.
Evaluate (epoch 7) -- MSE loss = 0.003303 +/- 0.001156
Adjusting learning rate of group 0 to 1.0000e-04.
Evaluate (epoch 8) -- MSE loss = 0.003390 +/- 0.001118
Adjusting learning rate of group 0 to 1.0000e-04.
Evaluate (epoch 9) -- MSE loss = 0.003418 +/- 0.001100
Adjus